In [ ]:
import altair as alt
import polars as pl


table = pl.read_csv("/master/abagwell/variant-analysis/results/marmoset/coverage/intersections/exons/exon_info.tsv", separator="\t", has_header=False, new_columns=["path", "length"]).with_columns(
    pl.col("path").str.split("/").list.get(0).alias("batch"),
    pl.col("path").str.split("/").list.get(1).alias("run"),
    (pl.col("length") / pl.lit(408227261)).alias("x_times_coverage"),
).with_columns(
    pl.col("run").str.split("_").list.get(0).alias("sample")
).drop("path")

In [ ]:
grouped_table = table.group_by("batch", "sample").agg(pl.sum("x_times_coverage"))

In [ ]:
#table.write_csv("/master/abagwell/figures/exome_coverage.tsv", separator="\t")

In [ ]:
grouped = grouped_table.group_by("batch").agg(pl.mean("x_times_coverage"), pl.count("sample"))


In [ ]:
alt.Chart(grouped_table).mark_boxplot().encode(
    alt.X("x_times_coverage:Q", title="X Times Coverage").scale(zero=False),
    alt.Y("batch:N", title="Batch"),
    tooltip = [
        alt.Tooltip("batch:N"),
        alt.Tooltip("sample:N"),
    ],
).properties(
    title = "Marmoset hWES Exome Coverage"
)#.save("/master/abagwell/figures/exome_coverage.group_by_batch.html")